In [44]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import os
import json
import numpy as np
import time
import random

In [48]:
random.uniform(1.0, 2.0)

1.8038719333039896

In [18]:
all_a = []
limit = 30
for i in tqdm(range(0, 100000, limit)):
    if i == 0:
        i = ''
    else:
        i = f'/+{i}'
    url = f'https://forum.lowyat.net/Kopitiam{i}'
    r = requests.get(url)
    soup = BeautifulSoup(r.content, "lxml")
    a = soup.find_all('a')
    a = [a_.get('href') for a_ in a if a_.get('href') and a_.get('href').startswith('/topic')]
    a = list(set([a_ for a_ in a if len(a_.split('/')) == 3]))
    if not len(a):
        break
    all_a.extend(a)

 16%|███████████████                                                                              | 541/3334 [07:23<38:11,  1.22it/s]


In [21]:
all_a = list(set(all_a))

In [22]:
len(all_a)

16214

In [24]:
with open('Kopitiam-topics.json', 'w') as fopen:
    json.dump(all_a, fopen)

In [28]:
from concurrent.futures import ThreadPoolExecutor, as_completed

def get_text(url):
    texts = []
    r_ = requests.get(url)
    soup = BeautifulSoup(r_.content, "lxml")
    divs = soup.find_all('div', {'class': 'post_text'})
    for div in divs:
        t = BeautifulSoup(str(div).replace('<br/>', '\n')).text.strip()
        if len(t):
            texts.append(t)
    return texts

In [34]:
topic[1:].replace('/', '-')

'topic-2872926'

In [49]:
base_dir

'topic-2872926'

In [56]:
last_texts = []
last_texts = [1][:]
last_texts

[1]

In [ ]:
limit = 20
worker_size = 1
per_batch = worker_size * limit

for topic in sorted(all_a):
    
    base_dir = topic[1:].replace('/', '-')
    os.makedirs(base_dir, exist_ok = True)
    os.makedirs('done', exist_ok = True)
    
    base_dir_filename = os.path.join('done', f'{base_dir}.json')
    
    if os.path.exists(base_dir_filename):
        continue
    
    need_break = False
    last_texts = []
    for i in tqdm(range(0, 100000, per_batch)):
        filename = os.path.join(base_dir, f'{i}.json')
        if os.path.exists(filename):
            continue
        aranged = np.arange(i, i + per_batch, limit)
        urls = []
        for a_ in aranged:
            if a_ == 0:
                a_ = ''
            else:
                a_ = f'/+{a_}'
            url = f'https://forum.lowyat.net/{topic}{a_}'
            urls.append(url)
        
        texts = []
        
        with ThreadPoolExecutor(max_workers=worker_size) as executor:
            futures = {executor.submit(get_text, item): item for item in urls}
            
            for future in as_completed(futures):
                r = future.result()
                texts.extend(r)
                if not len(r):
                    need_break = True
                    
                s = set(texts)
                if len(set(last_texts) & s) == len(s):
                    need_break = True
                    
                last_texts = texts[:]
        
        with open(filename, 'w') as fopen:
            json.dump(texts, fopen)
            
        time.sleep(random.uniform(0.5, 1.5))
        
        if need_break:
            break
    
    with open(base_dir_filename, 'w') as fopen:
        json.dump({'status': True}, fopen) 

  1%|█▏                                                                                              | 63/5000 [01:50<2:19:14,  1.69s/it]encoding error : input conversion failed due to input error, bytes 0x8D 0x6F 0x71 0x75
encoding error : input conversion failed due to input error, bytes 0x8D 0x6F 0x71 0x75
encoding error : input conversion failed due to input error, bytes 0x8D 0x6F 0x71 0x75
  1%|█▏                                                                                              | 64/5000 [01:53<2:42:12,  1.97s/it]encoding error : input conversion failed due to input error, bytes 0x8D 0x6F 0x71 0x75
encoding error : input conversion failed due to input error, bytes 0x8D 0x6F 0x71 0x75
encoding error : input conversion failed due to input error, bytes 0x8D 0x6F 0x71 0x75
  2%|██▏                                                                                            | 117/5000 [03:33<2:22:21,  1.75s/it]encoding error : input conversion failed due to input error, bytes 

  0%|▏                                                                                               | 13/5000 [00:21<2:20:26,  1.69s/it]encoding error : input conversion failed due to input error, bytes 0x8D 0x6F 0x71 0x75
encoding error : input conversion failed due to input error, bytes 0x8D 0x6F 0x71 0x75
encoding error : input conversion failed due to input error, bytes 0x8D 0x6F 0x71 0x75
  5%|████▋                                                                                          | 244/5000 [07:14<2:30:06,  1.89s/it]encoding error : input conversion failed due to input error, bytes 0x9E 0xD0 0xB1 0xD1
encoding error : input conversion failed due to input error, bytes 0x9E 0xD0 0xB1 0xD1
encoding error : input conversion failed due to input error, bytes 0x9E 0xD0 0xB1 0xD1
  5%|████▉                                                                                          | 258/5000 [07:40<2:37:51,  2.00s/it]encoding error : input conversion failed due to input error, bytes 

  0%|                                                                                                 | 1/5000 [00:02<3:01:19,  2.18s/it]encoding error : input conversion failed due to input error, bytes 0x8D 0x6F 0x71 0x75
encoding error : input conversion failed due to input error, bytes 0x8D 0x6F 0x71 0x75
encoding error : input conversion failed due to input error, bytes 0x8D 0x6F 0x71 0x75
  0%|                                                                                                 | 1/5000 [00:04<5:36:57,  4.04s/it]


  0%|                                                                                                 | 3/5000 [00:05<2:27:10,  1.77s/it]encoding error : input conversion failed due to input error, bytes 0x9E 0xD0 0xB1 0xD1
encoding error : input conversion failed due to input error, bytes 0x9E 0xD0 0xB1 0xD1
encoding error : input conversion failed due to input error, bytes 0x9E 0xD0 0xB1 0xD1
  0%|▎                                                                                               | 17/5000 [00:28<2:16:12,  1.64s/it]encoding error : input conversion failed due to input error, bytes 0x9E 0xD0 0xB1 0xD1
encoding error : input conversion failed due to input error, bytes 0x9E 0xD0 0xB1 0xD1
encoding error : input conversion failed due to input error, bytes 0x9E 0xD0 0xB1 0xD1
  0%|                                                                                                 | 5/5000 [00:09<2:36:45,  1.88s/it]encoding error : input conversion failed due to input error, bytes 

 14%|█████████████▏                                                                                 | 693/5000 [20:12<2:15:38,  1.89s/it]encoding error : input conversion failed due to input error, bytes 0x90 0x82 0xF0 0x9F
encoding error : input conversion failed due to input error, bytes 0x90 0x82 0xF0 0x9F
encoding error : input conversion failed due to input error, bytes 0x90 0x82 0xF0 0x9F
  0%|                                                                                                 | 1/5000 [00:01<2:11:49,  1.58s/it]encoding error : input conversion failed due to input error, bytes 0x8D 0x6F 0x71 0x75
encoding error : input conversion failed due to input error, bytes 0x8D 0x6F 0x71 0x75
encoding error : input conversion failed due to input error, bytes 0x8D 0x6F 0x71 0x75
  0%|                                                                                                 | 2/5000 [00:03<2:36:52,  1.88s/it]encoding error : input conversion failed due to input error, bytes 

  0%|▏                                                                                                | 8/5000 [00:17<3:07:08,  2.25s/it]


  0%|▏                                                                                               | 12/5000 [00:24<3:12:48,  2.32s/it]encoding error : input conversion failed due to input error, bytes 0x9D 0x20 0x53 0x65
encoding error : input conversion failed due to input error, bytes 0x9D 0x20 0x53 0x65
encoding error : input conversion failed due to input error, bytes 0x9D 0x20 0x53 0x65
  0%|                                                                                                           | 0/5000 [00:00<?, ?it/s]encoding error : input conversion failed due to input error, bytes 0x8D 0x6F 0x71 0x75
encoding error : input conversion failed due to input error, bytes 0x8D 0x6F 0x71 0x75
encoding error : input conversion failed due to input error, bytes 0x8D 0x6F 0x71 0x75
  0%|▏                                                                                                | 8/5000 [00:14<2:46:34,  2.00s/it]